In [294]:
# Controle data.gouv
# 1. caracteristics
import pandas as pd
import numpy as np

# Chemins d'accès aux données
repDataConcat = '../../dataConcat.gouv/'
repDataConsolidees = '../../dataConsolidees.gouv/'

# Liste des fichiers usagers disponibles
!pwd
!ls {repDataConcat}*  -lrt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


/Users/stephane/Documents/Professionnel/Formation/Certificats/Datascientest/4 - Projet AccidentsRoutiers/scripts/0 - dataConsolidation
ls: -lrt: No such file or directory
../../dataConcat.gouv/caracteristiques.csv
../../dataConcat.gouv/caracteristiques_2022.csv
../../dataConcat.gouv/lieux.csv
../../dataConcat.gouv/lieux_2022.csv
../../dataConcat.gouv/usagers copie.csv
../../dataConcat.gouv/usagers.csv
../../dataConcat.gouv/usagers_2022.csv
../../dataConcat.gouv/vehicules.csv
../../dataConcat.gouv/vehicules_2022.csv


#### PARTIE 1 - JOINTURE DES 4 FICHIERS (CARACTERISTIQUES + LIEUX + VEHICULE + USAGERS)
#### --------

0. Chargement des fichiers
1. Contrôle absence de doublons
2. Merge Caractéristiques et lieux
3. Ajout de véhicules
4. Ajout de usagers
5. Renommage des variables (ajout en préfixe de l'origine de la data (nom du fichier)
6. Ordonnancement des colonnes
7. Ajout d'une variable dichotomisée : vehi_catv_perso (regroupement perso des catégories de véhicules
8. Création du fichier csv


In [295]:
# I.0. Chargment des données des 4 fichiers
#suffixe = "_2022.csv"                # traitement du fichier consolidé sur 2022 uniquement
suffixe = ".csv"                      # traitement du fichier consolidé sur 2005 - 2022

lieux = pd.read_csv(repDataConcat+"lieux" + suffixe, dtype = {'voie' : str, 'pr': 'str', 'pr1' : str})
vehi = pd.read_csv(repDataConcat+"vehicules" + suffixe, dtype = {'id_vehicule' : int})
carac = pd.read_csv(repDataConcat+"caracteristiques" + suffixe, dtype = {'gps' : str})
usager = pd.read_csv(repDataConcat+"usagers" + suffixe, dtype = {'id_usager' : str, 'id_vehicule' : int, 'actp' : str})


In [296]:
# I.1. Controle absence de doublons sur les clés de chaque fichier
print("doublons de caracteristiques=", carac.duplicated(subset=['id_accident']).sum())
print("doublons de lieux=", lieux.duplicated(subset=['id_accident']).sum())
print("doublons de vehicule=", vehi.duplicated(subset=['id_accident', 'id_vehicule', 'num_veh']).sum())

# id_usager étant renseigné depuis 2021 --> la recherche de doublons n'a de sens qu'à partir de cette date
print("doublons de usager=",  usager.loc[usager['id_accident'].astype(str).str[:4] >= '2021'].duplicated(subset=['id_accident', 'id_vehicule', 'num_veh', 'id_usager']).sum())


doublons de caracteristiques= 0
doublons de lieux= 0
doublons de vehicule= 0
doublons de usager= 0


In [297]:
# I.2. Merge des caractéristiques avec les lieux en un seul df
df_merged = pd.merge(carac, lieux, on=['id_accident'])

# Contrôle
print ("nb d'enreg carac=", carac.shape) 
print ("nb d'enreg lieux=", lieux.shape) 
print ("nb d'enregistrement carac+lieux=", df_merged.shape)


nb d'enreg carac= (1176873, 16)
nb d'enreg lieux= (1176873, 19)
nb d'enregistrement carac+lieux= (1176873, 34)


In [298]:
# I.3. Ajout des véhicules
# Merge des caractéristiques avec les lieux en un seul df
df_merged = pd.merge(df_merged, vehi, on=['id_accident'])

# Contrôle
print ("nb d'enreg vehicules=", vehi.shape) 
print ("nb d'enregistrement carac+lieux+vehi=", df_merged.shape)
#df_merged = 


nb d'enreg vehicules= (2009395, 11)
nb d'enregistrement carac+lieux+vehi= (2009395, 44)


In [299]:
# I.4. Ajout des usagers
accident = pd.merge(df_merged, usager, on=['id_accident', 'id_vehicule', 'num_veh'])

# Contrôle
print ("nb d'enregistrement carac+lieux+vehi=", df_merged.shape)
print ("nb d'enreg usager=", usager.shape) 
print ("nb d'enregistrement carac+lieux+vehi+usager=", accident.shape)

accident.head()



nb d'enregistrement carac+lieux+vehi= (2009395, 44)
nb d'enreg usager= (2636377, 16)
nb d'enregistrement carac+lieux+vehi+usager= (2636355, 57)


,id_accident,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,gps_lat,gps_long,dept,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc,id_usager,place,catu,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp,gravite
0,200500000001,2005,1,12,1900,3,2,1,1,3,11.0,CD41B,M,50.515,2.944,59,3,41.0,0.0,B,2,2,41.0,41.0,0,1,1,0.0,63.0,1,0,1,0.0,NaN,0,A01,0.0,7,0,2,1,1,NaN,0,NaN,1,1,1,1976.0,1,1,-1,-1,0,0,0,1
1,200500000001,2005,1,12,1900,3,2,1,1,3,11.0,CD41B,M,50.515,2.944,59,3,41.0,0.0,B,2,2,41.0,41.0,0,1,1,0.0,63.0,1,0,1,0.0,NaN,0,B02,0.0,7,0,2,8,10,NaN,0,NaN,1,1,2,1968.0,3,1,-1,-1,0,0,0,2
2,200500000001,2005,1,12,1900,3,2,1,1,3,11.0,CD41B,M,50.515,2.944,59,3,41.0,0.0,B,2,2,41.0,41.0,0,1,1,0.0,63.0,1,0,1,0.0,NaN,0,B02,0.0,7,0,2,8,10,NaN,0,NaN,2,2,1,1964.0,-1,1,-1,-1,0,0,0,0
3,200500000001,2005,1,12,1900,3,2,1,1,3,11.0,CD41B,M,50.515,2.944,59,3,41.0,0.0,B,2,2,41.0,41.0,0,1,1,0.0,63.0,1,0,1,0.0,NaN,0,B02,0.0,7,0,2,8,10,NaN,0,NaN,4,2,1,2004.0,-1,3,-1,-1,0,0,0,0
4,200500000001,2005,1,12,1900,3,2,1,1,3,11.0,CD41B,M,50.515,2.944,59,3,41.0,0.0,B,2,2,41.0,41.0,0,1,1,0.0,63.0,1,0,1,0.0,NaN,0,B02,0.0,7,0,2,8,10,NaN,0,NaN,5,2,1,1998.0,-1,1,-1,-1,0,0,0,0


In [300]:
# I.5. Renommage des variables (rappel de l'origine de chaque colonne)
renamed_col = {
    'adr'     : 'carac_adr',
    'agg'     : 'carac_agg',
    'an'      : 'carac_an',
    'mois'    : 'carac_mois',
    'jour'    : 'carac_jour',
    'hrmn'    : 'carac_hrmn',    
    'atm'     : 'carac_atm',
    'col'     : 'carac_col',
    'com'     : 'carac_com',
    'dept'    : 'carac_dept',
    'gps'     : 'carac_gps', 
    'gps_lat' : 'carac_gps_lat',
    'gps_long' : 'carac_gps_long',
    'int'     : 'carac_int',    
    'lum'     : 'carac_lum',
    'catr'    : 'lieu_catr',
    'voie'    : 'lieu_voie',
    'v1'      : 'lieu_v1',
    'v2'      : 'lieu_v2',
    'circ'    : 'lieu_circ',
    'nbv'     : 'lieu_nbv',
    'pr'      : 'lieu_pr',
    'pr1'     : 'lieu_pr1',
    'vosp'    : 'lieu_vosp',
    'prof'    : 'lieu_prof',
    'plan'    : 'lieu_plan',
    'lartpc'  : 'lieu_lartpc',
    'larrout' : 'lieu_larrout',
    'surf'    : 'lieu_surf',
    'infra'   : 'lieu_infra',
    'situ'    : 'lieu_situ',
    'env1'    : 'lieu_env1',
    'vma'     : 'lieu_vma',
    'senc'    : 'vehi_senc',
    'catv'    : 'vehi_catv',
    'obs'     : 'vehi_obs',
    'obsm'    : 'vehi_obsm',
    'choc'    : 'vehi_choc',
    'manv'    : 'vehi_manv',
    'motor'   : 'vehi_motor',
    'occutc'  : 'vehi_occutc',
    'place'   : 'user_place',
    'catu'    : 'user_catu',
    'sexe'    : 'user_sexe',
    'an_nais' : 'user_an_nais',
    'trajet'  : 'user_trajet',
    'secu1'   : 'user_secu1',
    'secu2'   : 'user_secu2',
    'secu3'   : 'user_secu3',
    'locp'    : 'user_locp',
    'actp'    : 'user_actp',
    'etatp'   : 'user_etatp',
    'gravite' : 'user_gravite'
}

accident = accident.rename(columns=renamed_col)

In [301]:
# I.6. Remonte les clés en début de df
key_columns = ['id_accident', 'id_vehicule', 'num_veh', 'id_usager']

# Sélectionnez les colonnes qui ne sont pas réorganisées
other_columns = accident.columns.difference(key_columns).tolist()
other_columns.remove('user_gravite')

# Ordonne les clés en premier, le reste après, la target en dernier
ordered_columns = key_columns + other_columns + ['user_gravite']

accident = accident.reindex(columns=ordered_columns)


In [302]:
# I.7. DICHOTOMISATION DE CATV (regroupement perso en --> 6 classes)
# 01 - Vélo (01)
# 02 - Véhicule sans permis (02 - 03 - 30 - 31 - 32 - 35 - 36 - 41 - 42)
# 03 - Motocyclette (33 - 34 - 43)
# 04 - Véhicule léger (07 - 10)
# 05 - Poids lourd (13 - 14 - 15 - 16 - 17 - 20 - 37 - 38)
# 06 - Autres (00 - 21 - 39 - 40 - 50 - 60 - 80 - 99)

# 1. Remplacement des Nan par la classe la plus représentée
NaN_value = accident.vehi_catv.value_counts().index[0]
accident['vehi_catv'] = accident.vehi_catv.apply(lambda valeur :  NaN_value if valeur == -1 else valeur)

# 2. Aggrégation des valeurs
accident['agg_catv_perso'] = accident['vehi_catv'].apply(lambda vehicule : 1 if vehicule==1.0 else 
                                                                           2 if vehicule in (2.0, 3.0, 30.0, 31.0, 32.0, 35.0, 36.0, 41.0) else
                                                                           3 if vehicule in (33.0, 34.0, 43.0) else
                                                                           4 if vehicule in (7.0, 10.0) else
                                                                           5 if vehicule in (13.0, 14.0, 15.0, 16.0, 17.0, 20.0, 37.0, 38.0) else
                                                                           6)

# 3. Dichotomisation de l'information - pourra être additionné lors de l'aggrégation par accident
colonne_originale = accident['agg_catv_perso']
accident.drop([colonne for colonne in accident.columns if colonne.startswith('agg_catv_perso_')], axis=1, inplace=True) # Supprimer les colonnes déjà dichotomisées

accident = pd.get_dummies(accident, columns=['agg_catv_perso'])    

# 4. Transcode les colonnes dichotomisées de 'bool' --> 'int64'
for column in accident.filter(like='agg_catv_perso').columns :
    accident[column] = accident[column].astype(int)
    
accident['agg_catv_perso'] = colonne_originale                             # Si besoin de relancer la cellule

In [303]:
# I.8. Conserve le df --> usager (accidents maille usager)
usager = accident.copy()

usager.to_csv(repDataConsolidees + "usagers" + suffixe, index=False, encoding='utf-8')


#### PARTIE 2 - Aggrégation du fichier à la maille véhicule (cumul des infos d'usagers)
#### --------

1. Dichotomisation de la catégorie des usagers : conducteur - passager - piéton
2. Dichotomisation du genre des passagers --> conducteur/conductrice
3. Dichotomisation de la gravité de l'accident --> indemne / blessé léger / blessé hospitalisé / tué
4. Aggrégation afin de comptabiliser le nb de piéton, nb de passager, le genre de chaque conducteur
5. Ajout de ces nouvelles informations au  df 'usagers'
   

In [304]:
# II.1.  DICHOTOMISATION DE USER_CATU : Catégorie de l'usager
# 01 - Conducteur
# 02 - Passager
# 03 - Piéton
colonne_originale = accident['user_catu']
try:
    accident.drop(['user_catu_conducteur', 'user_catu_passager', 'user_catu_pieton', 'user_catu_trotinette'], axis=1, inplace=True) # Supprimer les colonnes déjà dichotomisées

except :
    None

# 1. Remplacement des NaN par la classe la plus représentée
NaN_value = accident.user_catu.value_counts().index[0]
accident['user_catu'] = accident.user_catu.apply(lambda valeur :  NaN_value if valeur == -1 else valeur)

# 2. Dichotomisation
accident = pd.get_dummies(accident, columns=['user_catu'], drop_first=False)           # ne supprime pas la colonne d'origine

# 3. Renommage des colonnes 
rename_columns = {'user_catu_1' : 'user_catu_conducteur',
                  'user_catu_2' : 'user_catu_passager',
                  'user_catu_3' : 'user_catu_pieton',
                  'user_catu_4' : 'user_catu_trotinette'}
accident = accident.rename(rename_columns, axis = 1)

# 4. Transcode les colonnes dichotomisées de 'bool' --> 'int64'
for column in accident.filter(like='user_catu').columns :
    accident[column] = accident[column].astype(int)

accident['user_catu'] = colonne_originale

                          

In [305]:
# II.2. DICHOTOMISATION DE USER_SEXE
# 01 - homme
# 02 - femme
colonne_originale = accident['user_sexe']
try:
    accident.drop(['user_sexe_homme', 'user_sexe_femme'], axis=1, inplace=True) # Supprimer les colonnes déjà dichotomisées

except :
    None

# 1. Remplacement des NaN par la classe la plus représentée
NaN_value = accident.user_sexe.value_counts().index[0]
accident['user_sexe'] = accident.user_sexe.apply(lambda valeur :  NaN_value if valeur == -1 else valeur)

# 2. Dichotomisation de la variable
accident = pd.get_dummies(accident, columns=['user_sexe'], drop_first=False)           # ne supprime pas la colonne d'origine
accident['user_sexe'] = colonne_originale

# 3. Renommage des colonnes 
rename_columns = {'user_sexe_1' : 'user_sexe_homme',
                  'user_sexe_2' : 'user_sexe_femme'}
accident = accident.rename(rename_columns, axis = 1)

# 4. Différenciation des conducteurs / conductrices
accident['is_conducteur'] = (accident['user_catu_conducteur'] & accident['user_sexe_homme']).astype(int)
accident['is_conductrice'] = (accident['user_catu_conducteur'] & accident['user_sexe_femme']).astype(int)
                          
                          

In [306]:
# II.3. DICHOTOMISATION DE USER_GRAV : Gravité de la blessure
# 01 - Indemne
# 02 - Tué
# 03 - Blessé
colonne_originale = accident['user_gravite']
try:
    accident.drop(['user_gravite_indemne', 'user_gravite_tue', 'user_gravite_blesse_leger', 'user_gravite_blesse_grave'], axis=1, inplace=True) # Supprimer les colonnes déjà dichotomisées
except:
    None

# 1. Remplacement des NaN par la classe la plus représentée
NaN_value = accident.user_gravite.value_counts().index[0]
accident['user_gravite'] = accident.user_gravite.apply(lambda valeur :  NaN_value if valeur == -1 else valeur)

# 2. Dichotomisation
accident = pd.get_dummies(accident, columns=['user_gravite'], drop_first=False)           # ne supprime pas la colonne d'origine

# 3. Renommage des colonnes 
rename_columns = {'user_gravite_0' : 'user_gravite_indemne',
                  'user_gravite_1' : 'user_gravite_blesse_leger',
                  'user_gravite_2' : 'user_gravite_blesse_grave',
                  'user_gravite_3' : 'user_gravite_tue'
                 }
accident = accident.rename(rename_columns, axis = 1)

#4. Transcode les colonnes dichotomisées de 'bool' --> 'int64'
for column in accident.filter(like='user_gravite').columns :
    accident[column] = accident[column].astype(int)

accident['user_gravite'] = colonne_originale

In [307]:
# II.4. Aggrégation des données à la maille véhicule
aggregation = {      
     # catégorie des usagers
     'is_conducteur': 'sum',
     'is_conductrice': 'sum', 
     'user_catu_pieton': 'sum', 
     'user_catu_passager': 'sum',
     # Gravité des blessures
     'user_gravite_indemne': 'sum',
     'user_gravite_blesse_leger': 'sum',
     'user_gravite_blesse_grave': 'sum',
     'user_gravite_tue': 'sum'
    }
agg_vehi = accident.groupby(['id_accident', 'id_vehicule', 'num_veh', 
                             'agg_catv_perso_1', 'agg_catv_perso_2',
                             'agg_catv_perso_3', 'agg_catv_perso_4', 
                             'agg_catv_perso_5', 'agg_catv_perso_6']).agg(aggregation).reset_index()

# Renommer les colonnes d'aggrégation
rename_columns = {
     # catégorie des véhicules
     'agg_catv_perso_1': 'agg_is_velo',
     'agg_catv_perso_2': 'agg_is_vsp',
     'agg_catv_perso_3': 'agg_is_moto',
     'agg_catv_perso_4': 'agg_is_vl',
     'agg_catv_perso_5': 'agg_is_pl',
     'agg_catv_perso_6': 'agg_is_va',        
     # catégorie des usagers
     'is_conducteur': 'agg_is_conducteur_vehicule',
     'is_conductrice': 'agg_is_conductrice_vehicule',
     'user_catu_pieton': 'agg_nb_pieton_vehicule', 
     'user_catu_passager': 'agg_nb_passager_vehicule',
     # Gravité des blessures
     'user_gravite_indemne': 'agg_nb_indemne_vehicule',
     'user_gravite_blesse_leger': 'agg_nb_blesse_leger_vehicule',
     'user_gravite_blesse_grave': 'agg_nb_blesse_grave_vehicule',
     'user_gravite_tue': 'agg_nb_tue_vehicule'
    }
agg_vehi.rename(columns=rename_columns, inplace=True)



In [308]:
# II.5. Consolidation df usager de ces nouvelles informations
usager = pd.merge(usager, agg_vehi, on=['id_accident', 'id_vehicule', 'num_veh'], how='outer')


In [309]:
# II.6. Suppression des colonnes inutiles de usager : agg_catv_perso_x)
prefix_cols_to_delete = 'agg_catv_perso_'
cols_to_delete = ['agg_is_velo', 'agg_is_vsp', 'agg_is_moto', 'agg_is_vl', 'agg_is_pl', 'agg_is_va']

# Filtrer les colonnes à supprimer en utilisant le préfixe
columns_to_delete = usager.filter(like=prefix_cols_to_delete).columns.tolist()
#columns_to_delete = columns_to_delete.tolist()
columns_to_delete.extend(cols_to_delete)

# Supprimer les colonnes
usager.drop(columns_to_delete, axis=1, inplace=True)


In [310]:
# II.6. Contrôles
#usager.info()
agg_vehi[(agg_vehi['id_accident'] == 200800000005)].head(50)
accident.loc[(accident['id_accident'] == 200800000005), ['vehi_catv', 'agg_catv_perso', 'agg_catv_perso_1', 
                                                         'agg_catv_perso_2', 'agg_catv_perso_3', 'agg_catv_perso_4']].head(50)
agg_vehi.head()

,id_accident,id_vehicule,num_veh,agg_is_velo,agg_is_vsp,agg_is_moto,agg_is_vl,agg_is_pl,agg_is_va,agg_is_conducteur_vehicule,agg_is_conductrice_vehicule,agg_nb_pieton_vehicule,agg_nb_passager_vehicule,agg_nb_indemne_vehicule,agg_nb_blesse_leger_vehicule,agg_nb_blesse_grave_vehicule,agg_nb_tue_vehicule
0,200500000001,0,A01,0,0,0,1,0,0,1,0,0,0,0,1,0,0
1,200500000001,0,B02,0,0,0,1,0,0,0,1,0,4,4,0,1,0
2,200500000002,0,A01,0,0,0,1,0,0,1,0,0,0,1,0,0,0
3,200500000002,0,B02,0,1,0,0,0,0,1,0,0,0,0,0,1,0
4,200500000003,0,A01,0,1,0,0,0,0,1,0,0,0,1,0,0,0


#### PARTIE 3 - Aggrégation maille accident
#### --------

1. aggrégation à l'accident --> nb_total_pieton, nb_total_conducteur/trice, nb_total_velo, etc...
2. Merge avec le df 'usager'
3. Contrôles
4. Sauvegarde du df 'usager avec toutes ces informations complémentaires
5. Sauvegarde du df agg_accident
   

In [311]:
# Ajout d'un id composé de id_vehicule et num_veh (pour compter le nombre de véhicule impacté dans un accident
agg_vehi['id_vehi'] = agg_vehi['id_vehicule'].astype(str) + ' - ' + agg_vehi['num_veh']
aggregation = {
     # nb de véhicules impliqués
     'id_vehi': 'nunique',
     # catégorie des véhicules
     'agg_is_velo': 'sum',
     'agg_is_vsp': 'sum',
     'agg_is_moto': 'sum',
     'agg_is_vl': 'sum',
     'agg_is_pl': 'sum',
     'agg_is_va': 'sum',        
     # catégorie des usagers
     'agg_is_conducteur_vehicule': 'sum',
     'agg_is_conductrice_vehicule': 'sum',
     'agg_nb_pieton_vehicule': 'sum', 
     'agg_nb_passager_vehicule': 'sum',
     # Gravité des blessures
     'agg_nb_indemne_vehicule': 'sum',
     'agg_nb_blesse_leger_vehicule': 'sum',
     'agg_nb_blesse_grave_vehicule': 'sum',
     'agg_nb_tue_vehicule': 'sum'
    }
agg_accident = agg_vehi.groupby('id_accident').agg(aggregation)

# Renommer les colonnes d'aggrégation
rename_columns = {
     'id_vehi': 'agg_nb_total_vehicule',
     # catégorie des véhicules
     'agg_is_velo': 'agg_nb_total_velo',
     'agg_is_vsp': 'agg_nb_total_vsp',
     'agg_is_moto': 'agg_nb_total_moto',
     'agg_is_vl': 'agg_nb_total_vl',
     'agg_is_pl': 'agg_nb_total_pl',
     'agg_is_va': 'agg_nb_total_va',        
     # catégorie des usagers
     'agg_is_conducteur_vehicule': 'agg_nb_total_conducteur',
     'agg_is_conductrice_vehicule': 'agg_nb_total_conductrice',
     'agg_nb_pieton_vehicule': 'agg_nb_total_pieton', 
     'agg_nb_passager_vehicule': 'agg_nb_total_passager',
     # Gravité des blessures
     'agg_nb_indemne_vehicule': 'agg_nb_total_indemne',
     'agg_nb_blesse_leger_vehicule': 'agg_nb_total_blesse_leger',
     'agg_nb_blesse_grave_vehicule': 'agg_nb_total_blesse_grave',
     'agg_nb_tue_vehicule': 'agg_nb_total_tue'
    }
agg_accident.rename(columns=rename_columns, inplace=True)
agg_accident.reset_index(inplace=True)


In [312]:
# III.2. Consolidation df usager avec ces nouvelles informations
usager = pd.merge(usager, agg_accident, on='id_accident', how='outer')


In [313]:
# III.3. Contrôles
accident.loc[(accident.id_accident == 200900000002), [ 'num_veh', 'id_vehicule']]

# Consolidation à la maille accident

accident.loc[(accident['id_accident'] == 200900000002), [ 'agg_catv_perso_1', 'agg_catv_perso_2',
'agg_catv_perso_3', 'agg_catv_perso_4', 'agg_catv_perso_5', 'agg_catv_perso_6']].head(10)

#accident.info()

accident[(accident['vehi_catv'] == 7) ].head()

,id_accident,id_vehicule,num_veh,id_usager,carac_adr,carac_agg,carac_an,carac_atm,carac_col,carac_com,carac_dept,carac_gps,carac_gps_lat,carac_gps_long,carac_hrmn,carac_int,carac_jour,carac_lum,carac_mois,lieu_catr,lieu_circ,lieu_env1,lieu_infra,lieu_larrout,lieu_lartpc,lieu_nbv,lieu_plan,lieu_pr,lieu_pr1,lieu_prof,lieu_situ,lieu_surf,lieu_v1,lieu_v2,lieu_vma,lieu_voie,lieu_vosp,user_actp,user_an_nais,user_etatp,user_locp,user_place,user_secu1,user_secu2,user_secu3,user_trajet,vehi_catv,vehi_choc,vehi_manv,vehi_motor,vehi_obs,vehi_obsm,vehi_occutc,vehi_senc,agg_catv_perso_1,agg_catv_perso_2,agg_catv_perso_3,agg_catv_perso_4,agg_catv_perso_5,agg_catv_perso_6,agg_catv_perso,user_catu_conducteur,user_catu_passager,user_catu_pieton,user_catu_trotinette,user_catu,user_sexe_homme,user_sexe_femme,user_sexe,is_conducteur,is_conductrice,user_gravite_indemne,user_gravite_blesse_leger,user_gravite_blesse_grave,user_gravite_tue,user_gravite
0,200500000001,0,A01,NaN,CD41B,2,2005,1,3,11.0,59,M,50.515,2.944,1900,1,12,3,1,3,2,0.0,0,63.0,0.0,2,1,41.0,41.0,1,1,1,0.0,B,NaN,41.0,0,0,1976.0,0,0,1,1,-1,-1,1,7,1,1,NaN,0,2,0,0.0,0,0,0,1,0,0,4,1,0,0,0,1,True,False,1,1,0,0,1,0,0,1
1,200500000001,0,B02,NaN,CD41B,2,2005,1,3,11.0,59,M,50.515,2.944,1900,1,12,3,1,3,2,0.0,0,63.0,0.0,2,1,41.0,41.0,1,1,1,0.0,B,NaN,41.0,0,0,1968.0,0,0,1,1,-1,-1,3,7,8,10,NaN,0,2,0,0.0,0,0,0,1,0,0,4,1,0,0,0,1,False,True,2,0,1,0,0,1,0,2
2,200500000001,0,B02,NaN,CD41B,2,2005,1,3,11.0,59,M,50.515,2.944,1900,1,12,3,1,3,2,0.0,0,63.0,0.0,2,1,41.0,41.0,1,1,1,0.0,B,NaN,41.0,0,0,1964.0,0,0,2,1,-1,-1,-1,7,8,10,NaN,0,2,0,0.0,0,0,0,1,0,0,4,0,1,0,0,2,True,False,1,0,0,1,0,0,0,0
3,200500000001,0,B02,NaN,CD41B,2,2005,1,3,11.0,59,M,50.515,2.944,1900,1,12,3,1,3,2,0.0,0,63.0,0.0,2,1,41.0,41.0,1,1,1,0.0,B,NaN,41.0,0,0,2004.0,0,0,4,3,-1,-1,-1,7,8,10,NaN,0,2,0,0.0,0,0,0,1,0,0,4,0,1,0,0,2,True,False,1,0,0,1,0,0,0,0
4,200500000001,0,B02,NaN,CD41B,2,2005,1,3,11.0,59,M,50.515,2.944,1900,1,12,3,1,3,2,0.0,0,63.0,0.0,2,1,41.0,41.0,1,1,1,0.0,B,NaN,41.0,0,0,1998.0,0,0,5,1,-1,-1,-1,7,8,10,NaN,0,2,0,0.0,0,0,0,1,0,0,4,0,1,0,0,2,True,False,1,0,0,1,0,0,0,0


In [314]:
# III.4. Réindexer les colonnes
ordre_colonnes = ['id_accident', 'id_vehicule', 'num_veh', 'id_usager', 
                  'carac_an',  'carac_mois', 'carac_jour', 'carac_hrmn', 'carac_adr', 'carac_agg', 'carac_atm', 'carac_col',  
                  'carac_com','carac_dept', 'carac_gps', 'carac_gps_lat', 'carac_gps_long', 'carac_int',  'carac_lum', 
                  'lieu_catr', 'lieu_circ', 'lieu_env1', 'lieu_infra', 'lieu_larrout', 'lieu_lartpc', 'lieu_nbv', 'lieu_plan', 
                  'lieu_pr', 'lieu_pr1', 'lieu_prof', 'lieu_situ', 'lieu_surf', 'lieu_v1', 'lieu_v2', 'lieu_vma', 'lieu_voie', 'lieu_vosp', 
                  'vehi_catv', 'vehi_choc', 'vehi_manv', 'vehi_motor','vehi_obs', 'vehi_obsm', 'vehi_occutc', 'vehi_senc',
                  'user_an_nais','user_catu', 'user_actp', 'user_etatp', 'user_locp', 'user_place', 'user_secu1', 'user_secu2', 
                  'user_secu3', 'user_sexe', 'user_trajet', 'user_gravite', 
                  'agg_catv_perso', 'agg_is_conducteur_vehicule', 'agg_is_conductrice_vehicule', 'agg_nb_pieton_vehicule',
                  'agg_nb_passager_vehicule', 'agg_nb_indemne_vehicule', 'agg_nb_blesse_leger_vehicule', 
                  'agg_nb_blesse_grave_vehicule', 'agg_nb_tue_vehicule', 'agg_nb_total_vehicule', 'agg_nb_total_velo', 
                  'agg_nb_total_vsp', 'agg_nb_total_moto', 'agg_nb_total_vl', 'agg_nb_total_pl', 'agg_nb_total_va', 
                  'agg_nb_total_conducteur', 'agg_nb_total_conductrice', 'agg_nb_total_pieton', 'agg_nb_total_passager', 
                  'agg_nb_total_indemne', 'agg_nb_total_blesse_leger', 'agg_nb_total_blesse_grave', 'agg_nb_total_tue' 
]
usager = usager.reindex(columns=ordre_colonnes) 


In [315]:
usager.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636355 entries, 0 to 2636354
Data columns (total 81 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id_accident                   int64  
 1   id_vehicule                   int64  
 2   num_veh                       object 
 3   id_usager                     object 
 4   carac_an                      int64  
 5   carac_mois                    int64  
 6   carac_jour                    int64  
 7   carac_hrmn                    int64  
 8   carac_adr                     object 
 9   carac_agg                     int64  
 10  carac_atm                     int64  
 11  carac_col                     int64  
 12  carac_com                     float64
 13  carac_dept                    int64  
 14  carac_gps                     object 
 15  carac_gps_lat                 float64
 16  carac_gps_long                float64
 17  carac_int                     int64  
 18  carac_lum             

In [316]:
# III.4. Sauvegarde fichier usagers.csv et accidents.csv (aggréation de usagers)
usager.to_csv(repDataConsolidees + "agg_usagers" + suffixe, index=False, encoding='utf-8')

# Fichier accident aggrégé
agg_accident.to_csv(repDataConsolidees + "agg_accidents" + suffixe, index=False, encoding='utf-8')

In [293]:
# Controle la liste de accident ayant une incohérence sur 'occutc' identifiée dans traitement concaténation de vehicule
# A vérifier ici que le nombre de passage de chacun des véhicule est inférieur à 5 afin d'écarter la probabilité 
# Qu'il s'agisse d'un véhicule de transport en commun mal cétégorisé comme pourrait le faire croire la valeur de occutc.
usager[(usager.id_accident.isin([200500005639,
 200600002948,
 200600003484,
 200600006532,
 200600016249,
 200600016714,
 200600018630,
 200600019093,
 200600019868,
 200600020210,
 200600020257,
 200600020297,
 200600020455,
 200600020768,
 200600020838,
 200600021084,
 200600021774,
 200600021774,
 200600021995,
 200600022220,
 200600022359,
 200600022421,
 200600022422,
 200600022431,
 200600022436,
 200600022443,
 200600022482,
 200600022492,
 200600022821,
 200600022916,
 200600023266,
 200600023709,
 200600026755,
 200600026755,
 200600026791,
 200600026791,
 200600026802,
 200600027590,
 200600028486,
 200600029115,
 200600029162,
 200600029170,
 200600029268,
 200600029275,
 200600029820,
 200600029821,
 200600030496,
 200600031873,
 200600033905,
 200600036731,
 200600036749,
 200600038729,
 200600043113,
 200600044029,
 200600046757,
 200600047627,
 200600058730,
 200600075391,
 200600075392,
 200700001418,
 200700007860,
 200700010267,
 200700038698,
 200700045781,
 200700055724,
 200700062176,
 200700066876,
 200700073231,
 200700079884,
 200700081187,
 200700082772,
 200800005515,
 200800007558,
 200800014784,
 200800023411,
 200800027244,
 200800038443,
 200800038443,
 200800038444,
 200800038449,
 200800053312,
 200800053312,
 200800066828,
 200800067366,
 200800067933]))]

,id_accident,id_vehicule,num_veh,id_usager,carac_an,carac_mois,carac_jour,carac_hrmn,carac_adr,carac_agg,carac_atm,carac_col,carac_com,carac_dept,carac_gps,carac_gps_lat,carac_gps_long,carac_int,carac_lum,lieu_catr,lieu_circ,lieu_env1,lieu_infra,lieu_larrout,lieu_lartpc,lieu_nbv,lieu_plan,lieu_pr,lieu_pr1,lieu_prof,lieu_situ,lieu_surf,lieu_v1,lieu_v2,lieu_vma,lieu_voie,lieu_vosp,vehi_catv,vehi_choc,vehi_manv,vehi_motor,vehi_obs,vehi_obsm,vehi_occutc,vehi_senc,user_an_nais,user_catu,user_actp,user_etatp,user_locp,user_place,user_secu1,user_secu2,user_secu3,user_sexe,user_trajet,user_gravite,agg_catv_perso,agg_is_conducteur_vehicule,agg_is_conductrice_vehicule,agg_nb_pieton_vehicule,agg_nb_passager_vehicule,agg_nb_indemne_vehicule,agg_nb_blesse_leger_vehicule,agg_nb_blesse_grave_vehicule,agg_nb_tue_vehicule,agg_nb_total_vehicule,agg_nb_total_velo,agg_nb_total_vsp,agg_nb_total_moto,agg_nb_total_vl,agg_nb_total_pl,agg_nb_total_va,agg_nb_total_conducteur,agg_nb_total_conductrice,agg_nb_total_pieton,agg_nb_total_passager,agg_nb_total_indemne,agg_nb_total_blesse_leger,agg_nb_total_blesse_grave,agg_nb_total_tue
12952,200500005639,0,A01,NaN,2005,1,25,835,HENRI CHARON (AVENUE),2,9,3,657.0,91,NaN,NaN,NaN,2,1,2,1,0.0,0,600.0,0.0,0,1,0.0,0.0,1,1,1,0.0,NaN,NaN,0.0,0,7,1,20,NaN,0,9,0,0.0,1960.0,1,0,0,0,1,1,-1,-1,2,1,0,4,0,1,0,0,1,0,0,0,2,1,0,0,1,0,0,0,2,0,0,1,1,0,0
12953,200500005639,0,B01,NaN,2005,1,25,835,HENRI CHARON (AVENUE),2,9,3,657.0,91,NaN,NaN,NaN,2,1,2,1,0.0,0,600.0,0.0,0,1,0.0,0.0,1,1,1,0.0,NaN,NaN,0.0,0,1,7,16,NaN,0,2,0,0.0,1973.0,1,0,0,0,1,2,-1,-1,2,1,1,1,0,1,0,0,0,1,0,0,2,1,0,0,1,0,0,0,2,0,0,1,1,0,0
203864,200600002948,0,A01,NaN,2006,1,14,2000,MARC (BD NOEL),2,1,7,15.0,78,NaN,NaN,NaN,1,5,4,2,0.0,0,0.0,2.0,2,1,0.0,0.0,1,1,1,0.0,NaN,NaN,0.0,0,31,1,1,NaN,0,2,0,0.0,1953.0,1,0,0,0,1,2,-1,-1,1,-1,2,2,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0
205048,200600003484,0,A01,NaN,2006,1,17,1320,LOUIS PASTEUR,2,3,3,64.0,92,NaN,NaN,NaN,3,1,4,3,99.0,0,130.0,2.0,2,0,0.0,0.0,1,1,2,0.0,NaN,NaN,0.0,0,34,2,10,NaN,0,0,0,1.0,1988.0,2,0,0,0,0,8,-1,-1,1,-1,1,3,0,0,0,1,0,1,0,0,2,0,0,1,1,0,0,0,1,0,1,1,1,0,0
205049,200600003484,0,B01,NaN,2006,1,17,1320,LOUIS PASTEUR,2,3,3,64.0,92,NaN,NaN,NaN,3,1,4,3,99.0,0,130.0,2.0,2,0,0.0,0.0,1,1,2,0.0,NaN,NaN,0.0,0,7,3,15,NaN,0,2,0,0.0,1947.0,1,0,0,0,1,8,-1,-1,2,1,0,4,0,1,0,0,1,0,0,0,2,0,0,1,1,0,0,0,1,0,1,1,1,0,0
212093,200600006532,0,A01,NaN,2006,2,16,1345,NaN,1,2,3,225.0,12,M,0.000,0.000,1,1,3,2,0.0,0,75.0,2.0,2,2,992.0,992.0,2,1,2,0.0,NaN,NaN,992.0,0,7,7,13,NaN,0,2,0,0.0,1982.0,1,0,0,0,1,1,-1,-1,2,1,3,4,0,1,0,0,0,0,0,1,2,0,0,0,2,0,0,1,1,0,3,1,3,0,1
212094,200600006532,0,B02,NaN,2006,2,16,1345,NaN,1,2,3,225.0,12,M,0.000,0.000,1,1,3,2,0.0,0,75.0,2.0,2,2,992.0,992.0,2,1,2,0.0,NaN,NaN,992.0,0,10,1,1,NaN,0,2,0,0.0,1937.0,1,0,0,0,1,1,-1,-1,1,4,0,4,1,0,0,3,1,3,0,0,2,0,0,0,2,0,0,1,1,0,3,1,3,0,1
212095,200600006532,0,B02,NaN,2006,2,16,1345,NaN,1,2,3,225.0,12,M,0.000,0.000,1,1,3,2,0.0,0,75.0,2.0,2,2,992.0,992.0,2,1,2,0.0,NaN,NaN,992.0,0,10,1,1,NaN,0,2,0,0.0,1968.0,2,0,0,0,2,0,-1,-1,2,-1,1,4,1,0,0,3,1,3,0,0,2,0,0,0,2,0,0,1,1,0,3,1,3,0,1
212096,200600006532,0,B02,NaN,2006,2,16,1345,NaN,1,2,3,225.0,12,M,0.000,0.000,1,1,3,2,0.0,0,75.0,2.0,2,2,992.0,992.0,2,1,2,0.0,NaN,NaN,992.0,0,10,1,1,NaN,0,2,0,0.0,1964.0,2,0,0,0,4,0,-1,-1,2,-1,1,4,1,0,0,3,1,3,0,0,2,0,0,0,2,0,0,1,1,0,3,1,3,0,1
212097,200600006532,0,B02,NaN,2006,2,16,1345,NaN,1,2,3,225.0,12,M,0.000,0.000,1,1,3,2,0.0,0,75.0,2.0,2,2,992.0,992.0,2,1,2,0.0,NaN,NaN,992.0,0,10,1,1,NaN,0,2,0,0.0,1958.0,2,0,0,0,3,0,-1,-1,2,-1,1,4,1,0,0,3,1,3,0,0,2,0,0,0,2,0,0,1,1,0,3,1,3,0,1
